In [1]:
#pragma cling add_include_path("../../include")
#pragma cling add_include_path("/usr/include/jsoncpp")
#pragma cling add_include_path("../feltor/inc") // Feltor path
#pragma cling load("jsoncpp")
#define THRUST_DEVICE_SYSTEM THRUST_DEVICE_SYSTEM_CPP
#include <iostream>
#include "dg/algorithm.h"
#include "dg/file/json_utilities.h"

In file included from input_line_8:2:
In file included from ../feltor/inc/dg/algorithm.h:8:
../feltor/inc/dg/backend/config.h:20:9: warning: NOTE: Fast std::fma(a,b,c) not activated! Using a*b+c instead! [-W#pragma-messages]
#pragma message( "NOTE: Fast std::fma(a,b,c) not activated! Using a*b+c instead!")
        ^
In file included from input_line_8:2:
In file included from ../feltor/inc/dg/algorithm.h:11:
In file included from ../feltor/inc/dg/topology/split_and_join.h:4:
In file included from ../feltor/inc/dg/backend/blas1_dispatch_shared.h:12:
In file included from ../feltor/inc/dg/backend/blas1_serial.h:6:
In file included from ../feltor/inc/dg/backend/exblas/exdot_serial.h:25:
In file included from ../feltor/inc/dg/backend/exblas/accumulate.h:19:
../feltor/inc/dg/backend/exblas/config.h:31:9: warning: WARNING: Instruction set below SSE4.1! Deactivating vectorization!
      [-W#pragma-messages]
#pragma message("WARNING: Instruction set below SSE4.1! Deactivating vectorization!")
 

# JSON utilities

We use the [jsoncpp](https://github.com/open-source-parsers/jsoncpp) library to parse json files. We chose it because it is widely used, very well maintained and even available as a linux package `libjsoncpp`.

## The `dg::file::file2Json` function
The first simplification regards how comments and errors are treated on file opening. We provide a shortcut:

In [9]:
#include "dg/file/json_utilities.h"
dg::file::WrappedJsonValue js;
try{
    dg::file::file2Json( "does_not_exist.json", js.asJson(),
                    dg::file::comments::are_discarded,
                    dg::file::error::is_throw);
}catch( std::exception& e)
{
    std::cout << e.what();
}


An error occured while parsing does_not_exist.json
*** File does not exist! *** 



In this example we open a file where C-style comments are allowed but discarded, while an error on opening or reading the file leads to a throw (which if not captured leads to immediate abortion of the program).

## The `dg::file::WrappedJsonValue` class
For our purposes the only downside of jsoncpp is that missing values do not trigger a throw and to manually check existence somewhat clutters the code. At the same time missing values often result from silly mistakes, which in a high performance computing environment result in real avoidable cost (for example because you ran a simulation with a default value that you did not really intend to).

For this reason we provide the `dg::file::WrappedJsonValue` class.
It basically wraps the
access to a `Json::Value` with guards that raise exceptions or display warnings in case an error occurs, for example when a key is misspelled,
missing or has the wrong type.
The goal is the composition of a good error message that helps a user
quickly debug the input file.

The Wrapper is necessary because Jsoncpp by default silently
generates a new key in case it is not present which in our scenario is an
invitation for stupid mistakes.
You can use the `WrappedJsonValue` like a `Json::Value` with read-only access:

In [3]:
dg::file::WrappedJsonValue ws;
try{
    dg::file::file2Json( "test.json", ws.asJson(),
                    dg::file::comments::are_discarded,
                    dg::file::error::is_throw);
}catch( std::exception& e){ std::cerr << e.what();}

In [8]:
try{
    std::string hello = ws[ "does not exist"].asString();
} catch ( std::exception& e){
    std::cout << "Error in file test.json\n";
    std::cout << e.what()<<std::endl;
}

Error in file test.json
*** Key error: "does not exist":  not found.


A feature of the class is that it keeps track of how a value is called.
For example

In [5]:
void some_function( dg::file::WrappedJsonValue ws)
{
    int value = ws[ "some_non_existent_key"].asUInt();
    std::cout << value<<"\n";
}

In [6]:
try{
    some_function( ws["grid"]);
} catch ( std::exception& e){ std::cout << e.what()<<std::endl; }


*** Key error: "grid": "some_non_existent_key":  not found.


The what string knows that "some_non_existent_key" is expected to be
contained in the "grid" key, which simplifies debugging to a great extent.